In [1]:
import csv
import requests
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np

In [35]:
# Specify the path to the larger folder containing the participant data
folder_path = "C:/Users/Jasmine Awad/Desktop/COMS Data Science/RawDataASLTB"

# Define the participant ID (I do this for each participant to check for errors)
participant_list = os.listdir(folder_path)


# define correct thingys
CorrRespST = ['brother and wife (sister in law)', 
              'using the restroom makes noise', 
              'ASL instructor', 
              'Bangladesh', 
              'Australia', 
              'Date cookie']
CorrRespMaze = {'ASLCompStim1.jpg':'["purse","store","balloon"]', 
                'ASLCompStim2.jpg':'["flower","dog","balloon"]', 
                'ASLCompStim3.jpg':'["bike","balloon","hospital"]',
                'ASLCompStim4.jpg':'["purse","hospital","store"]', 
                'ASLCompStim5.jpg':'["dog","store","bike"]', 
                'ASLCompStim6.jpg':'["flower","store","balloon"]'}


In [31]:
print(participant_list)

['C01', 'C02', 'C05', 'C08', 'C09', 'C10', 'C11', 'C17', 'CleanedData', 'D02', 'D03', 'D05', 'D06', 'H02', 'H03', 'H05', 'H07', 'H08', 'H09', 'H12', 'H13', 'H14', 'H15', 'H18', 'S02', 'S03', 'S04', 'S05', 'S06', 'S07', 'S08', 'S09', 'S11', 'T04', 'T05', 'T08']


In [60]:
for participant_id in participant_list:
    # Construct the path to the participant's folder
    participant_folder = os.path.join(folder_path, participant_id)
    
    # Check if the participant's folder exists
    if os.path.exists(participant_folder):
        # Get a list of files in the participant's folder
        files = os.listdir(participant_folder)


        # Filter out any subdirectories from the list of files
        raw_files = [file for file in files if os.path.isfile(os.path.join(participant_folder, file))]

        # Flag to indicate if the ASLTestBattery1 file is found
        found_asl_test_battery1 = False
        # Flag to indicate if the ASLTestBattery2 file is found
        found_asl_test_battery2 = False

        # Dataframes to store the cleaned data
        cleaned_data1 = None
        cleaned_data2 = None

        # Process each raw data file
        for raw_file in raw_files:
            # Check if the current file is the "ASLTestBattery1" file
            if "ASLTestBattery1" in raw_file:
                found_asl_test_battery1 = True

                # Construct the path to the raw data file
                file_path = os.path.join(participant_folder, raw_file)

                # Load the raw data file into a pandas DataFrame
                df = pd.read_csv(file_path)

                # Extract the columns of interest from ASLTestBattery1
                typed_word = df["typedWord"]
                item_prompt = df["itemPrompt"]
                image_file = df["imageFile"]
                clicked_response = df["mouse_2.clicked_name"]

                # Remove blank rows from the imageFile column and reset index
                image_file = image_file.dropna().reset_index(drop=True)

                # Remove blank rows from the mouse_2.clicked_name column and reset index
                clicked_response = clicked_response.dropna().reset_index(drop=True)

                # Perform data cleaning or any other desired operations on the extracted columns
                # ...

                # Combine the extracted columns into a DataFrame
                cleaned_data1 = pd.DataFrame({"typedWord": typed_word, "itemPrompt": item_prompt, "imageFile": image_file, "mouse_2.clicked_name": clicked_response})

            # Check if the current file is the "ASLTestBattery2" file
            if "ASLTestBattery2" in raw_file:
                found_asl_test_battery2 = True

                # Construct the path to the raw data file
                file_path = os.path.join(participant_folder, raw_file)

                # Load the raw data file into a pandas DataFrame
                df = pd.read_csv(file_path)

                # Extract the columns of interest from ASLTestBattery2
                typed_word2 = df["typedWord"]
                FSvideo_file = df["videoFile"][:10]  # Extract the first 10 rows
                Matchvideo_file = df["videoFile"][10:] #extract the rest
                MatchResponse = df["resp.clicked_name"]
                Match_CorrAns = df["corrAns"]

                # Remove blank rows from the match Response column and reset index
                MatchResponse = MatchResponse.dropna().reset_index(drop=True)

                # Remove blank rows from the image drawn column and reset index
                Image_Drawn = Image_Drawn.dropna().reset_index(drop=True)

                # Remove blank rows from the corrAns column and reset index
                Match_CorrAns = Match_CorrAns.dropna().reset_index(drop=True)

                # Perform data cleaning or any other desired operations on the extracted columns
                # ...

                # Combine the extracted columns into a DataFrame
                cleaned_data2 = pd.DataFrame({"typedWord2": typed_word2, "videoFile": FSvideo_file,
                                              "matchResponse": MatchResponse, "imageDrawn": Image_Drawn,
                                              "matchCorrAns": Match_CorrAns})

        # Combine the cleaned data from ASLTestBattery1 and ASLTestBattery2, if available
    if cleaned_data1 is not None and cleaned_data2 is not None:
        cleaned_data = pd.concat([cleaned_data1, cleaned_data2], axis=1)

        # Add the additional columns with blank values
        num_rows = len(cleaned_data)

        # Create the columns with NaN values
        cleaned_data['CorrRespST'] = np.nan
        cleaned_data['CorrRespMaze'] = np.nan
        cleaned_data['MatchCorrect'] = np.nan

        # Assign the values to the respective columns
        cleaned_data.loc[:len(CorrRespST)-1, 'CorrRespST'] = CorrRespST
        for img in CorrRespMaze:
            cleaned_data.loc[cleaned_data['imageFile'] == img, 'CorrRespMaze'] = CorrRespMaze[img]
        for i in range(len(MatchResponse)):
            if MatchResponse[i] == Match_CorrAns[i]:
                cleaned_data.loc[i, 'MatchCorrect'] = 1
            else:
                cleaned_data.loc[i, 'MatchCorrect'] = 0

        # Save the cleaned data to a new file
        cleaned_data.to_csv(folder_path + "/CleanedData/" + participant_id + '_cleaned.csv', index=False)

        # Print a message indicating successful completion
        print(f"Data processing and column addition completed successfully for {participant_id}.")
    else:
        print(f"{participant_id} is a bad person and they should feel bad!!")



Data processing and column addition completed successfully for C01.
Data processing and column addition completed successfully for C02.
Data processing and column addition completed successfully for C05.
Data processing and column addition completed successfully for C08.
Data processing and column addition completed successfully for C09.
Data processing and column addition completed successfully for C10.
Data processing and column addition completed successfully for C11.
Data processing and column addition completed successfully for C17.
CleanedData is a bad person and they should feel bad!!
Data processing and column addition completed successfully for D02.
Data processing and column addition completed successfully for D03.
Data processing and column addition completed successfully for D05.
Data processing and column addition completed successfully for D06.
Data processing and column addition completed successfully for H02.
Data processing and column addition completed successfully fo